**Pacotes necessários**

In [ ]:
import pandas as pd

**Importar dados de interesse**

**Dados ENEM 2019 e dicionário com descrição de informações de interesse**

Fonte: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem

In [ ]:
deletar = ['NU_ANO','TP_STATUS_REDACAO','NU_NOTA_COMP1', 'NU_NOTA_COMP2',
           'NU_NOTA_COMP3', 'NU_NOTA_COMP4','NU_NOTA_COMP5','TX_RESPOSTAS_CN', 
           'TX_RESPOSTAS_CH', 'TX_RESPOSTAS_LC', 'TX_RESPOSTAS_MT', 'TX_GABARITO_CN',
           'TX_GABARITO_CH', 'TX_GABARITO_LC', 'TX_GABARITO_MT', 'Q003', 'Q004', 'Q005',
           'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014', 'Q015', 'Q016',
           'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022', 'Q023', 'Q024']

In [ ]:
df = pd.read_csv('DADOS/MICRODADOS_ENEM_2019.csv', sep=';', encoding='latin-1')
df.drop(deletar, axis=1, inplace=True)
df

In [ ]:
dicionario = pd.read_excel('dicionario.xlsx')
dicionario.head(2)

**Dados longitudinais**

Fonte: https://github.com/kelvins/Municipios-Brasileiros

In [ ]:
mapa = pd.read_csv('Municipios-Brasileiros/csv/municipios.csv', sep=',')
mapa = mapa[['codigo_ibge','latitude','longitude']]
mapa.head()

**Geolocalizando dados do ENEM**

In [ ]:
df.isna().sum()

---

+3 milhoes de infos sem resposta para local da escola

Usar local da prova para informação geográfica

---

In [ ]:
geo = df[['CO_MUNICIPIO_PROVA', 'CO_MUNICIPIO_ESC']]
print('nan antes de drop:', geo.isna().sum())
geo = geo.dropna()
print('nan depois de drop:', geo.isna().sum())

In [ ]:
# Verificar taxa de diferença
geo.CO_MUNICIPIO_ESC = geo.CO_MUNICIPIO_ESC.astype(int)
geo.dtypes

In [ ]:
geo

In [ ]:
geo['validate'] = geo.CO_MUNICIPIO_PROVA - geo.CO_MUNICIPIO_ESC
geo

In [ ]:
geo.shape

In [ ]:
geo[geo.validate==0].shape

In [ ]:
print('proporção de coincidencia de mun. de residencia e mun. prova:',
      geo[geo.validate==0].shape[0]/geo.shape[0])

Assim sendo, do que foi preenchido para mun de situação da escola do aluno, cerca de 80% coincide com o municipio em que o aluno fez a prova. Toma-se como razoável nesta análise considerar a nota do aluno como sendo do munícipio em que o mesmo realizou a prova para análise geográfica distributiva.

In [ ]:
df = df.merge(mapa, how='left', left_on='CO_MUNICIPIO_PROVA', right_on='codigo_ibge')
df

**Trazendo informações descritivas de colunas de interesse**

In [ ]:
idade = dict([(i,a) for i,a in zip(dicionario['TP_FAIXA_ETARIA'].dropna(), dicionario['TP_FAIXA_ETARIA_DEF'])])
civil = dict([(i,a) for i,a in zip(dicionario['TP_ESTADO_CIVIL'].dropna(), dicionario['TP_ESTADO_CIVIL_DEF'])])
cor = dict([(i,a) for i,a in zip(dicionario['TP_COR_RACA'].dropna(), dicionario['TP_COR_RACA_DEF'])])
nacionalidade = dict([(i,a) for i,a in zip(dicionario['TP_NACIONALIDADE'].dropna(), dicionario['TP_NACIONALIDADE_DEF'])])
internet = dict([(i,a) for i,a in zip(dicionario['Q025'].dropna(), dicionario['ACESSO_INTERNET'])])
serie_pai = dict([(i,a) for i,a in zip(dicionario['Q001'].dropna(), dicionario['SERIE_PAI'])])
serie_mae = dict([(i,a) for i,a in zip(dicionario['Q002'].dropna(), dicionario['SERIE_MAE'])])
renda = dict([(i,a) for i,a in zip(dicionario['Q006'].dropna(), dicionario['RENDA_FAMILIA'])])

In [ ]:
df['FAIXA_ETARIA'] = [idade[valor] for valor in df.TP_FAIXA_ETARIA]
df['ESTADO_CIVIL'] = [civil[valor] for valor in df.TP_ESTADO_CIVIL]
df['COR_RACA'] = [cor[valor] for valor in df.TP_COR_RACA]
df['NACIONALIDADE'] = [nacionalidade[valor] for valor in df.TP_NACIONALIDADE]
df['ACESSO_INTERNET'] = [internet[valor] for valor in df.Q025]
df['SERIE_PAI'] = [serie_pai[valor] for valor in df.Q001]
df['SERIE_MAE'] = [serie_mae[valor] for valor in df.Q002]
df['RENDA_FAMILIAR'] = [renda[valor] for valor in df.Q006]

In [ ]:
df.columns

In [ ]:
del df['TP_FAIXA_ETARIA']
del df['TP_ESTADO_CIVIL']
del df['TP_COR_RACA']
del df['TP_NACIONALIDADE']
del df['Q025']
del df['Q001']
del df['Q002']
del df['Q006']
del df['codigo_ibge']
del df['CO_MUNICIPIO_ESC']
del df['NO_MUNICIPIO_ESC']
del df['CO_UF_ESC']
del df['SG_UF_ESC']
del df['TP_DEPENDENCIA_ADM_ESC']
del df['TP_LOCALIZACAO_ESC']
del df['TP_SIT_FUNC_ESC']

In [ ]:
df.columns

In [ ]:
df.CO_PROVA_MT

In [ ]:
df.to_csv('dados_pbi.csv', index=False)